In [1]:
library("rwwa")

Loading required package: extRemes

Loading required package: Lmoments

Loading required package: distillery


Attaching package: ‘extRemes’


The following objects are masked from ‘package:stats’:

    qqnorm, qqplot


Loading required package: plyr


Attaching package: ‘plyr’


The following object is masked from ‘package:distillery’:

    is.formula


Loading required package: copula

Loading required package: MASS



In [6]:
varnm <- "tasmin"
nsamp <- 1000
rp <- 3

fpath = paste0("~/00_WWA_project_folder/ephemeral/Central-asia-heat/highresmip/dat-",varnm,"/")
fl = list.files(fpath, full.names = F)

In [7]:
for(fnm in fl) {
    
    gcm <- strsplit(fnm, "_")[[1]][2]
    print(gcm)

    res_fnm <- paste0("~/01_wwa/10_misc/c-asia-res/res_",varnm,"_",gcm,".csv")
    if(!file.exists(res_fnm)) {
        
        gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/highresmip/GSAT/", pattern = paste0(gcm,"_"))
        if(length(gmst_fnm) == 0) { next }
        
        # combine into single dataframe
        gmst <- load_ts(paste0("~/00_WWA_project_folder/live/data/highresmip/GSAT/", gmst_fnm[1]), col.names = c("year", "gmst"))
        df <- merge(gmst, load_ts(paste0(fpath,fnm), col.names = c("year", varnm)))
        mdl <- fit_ns("gev", "shift", df, varnm, c("gmst"), lower = F)
        
        cov_now <- gmst[gmst$year == 2025,,drop = F]
        cov_hist <- cov_now - 1.3
        cov_fut <- cov_now + 1.3

        # bootstrap results
        res <- cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut, 
                              y_now = 2025, y_start = 1950, y_fut = 2070, nsamp = nsamp, di_relative = F)
        write.csv(res, res_fnm)
        
        # quick return level plots to check fit
        rlplot_fnm <- gsub("csv", "png", gsub("res/", "res/rlplot_", res_fnm))
        if(!file.exists(rlplot_fnm)) {
            png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
                plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = gsub("ts/", "", gsub(".csv", "", fnm)))
            }; dev.off()
        }
    }
}

[1] "CNRM-CM6-1"
[1] "CNRM-CM6-1-HR"
[1] "EC-Earth3P"
[1] "EC-Earth3P-HR"
[1] "FGOALS-f3-L"
[1] "HadGEM3-GC31-HM"
[1] "HadGEM3-GC31-LM"
[1] "HadGEM3-GC31-MM"
[1] "HiRAM-SIT-HR"
[1] "HiRAM-SIT-LR"
[1] "MPI-ESM1-2-HR"
[1] "MPI-ESM1-2-XR"
[1] "NICAM16-7S"
[1] "NICAM16-8S"


In [8]:
# compile results
res <- t(sapply(list.files("~/01_wwa/10_misc/c-asia-res", pattern = paste0("^res_",varnm), full.names = T), read.csv, row.names = "X"))
rownames(res) <- sapply(strsplit(gsub(".csv", "", rownames(res)), "_"), "[", 5)
write.csv(res, paste0("res-models_highresSST_",varnm,"_gmst.csv"))